In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [62]:

# Load datasets
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
ratings = pd.read_csv('ratings.csv')
orders = pd.read_csv('orders.csv')

#  # Check if customer_id 2 exists in the orders table
# customer_id = 2

# # customer_exists = customer_id in orders['customer_id'].values

# customer_orders = orders[orders['customer_id'] == customer_id]

# # Print the result
# if not customer_orders.empty:
#     print(f"Orders for Customer ID {customer_id}:")
#     print(customer_orders)
# else:
#     print(f"Customer ID {customer_id} does not exist in the orders table.")

In [4]:
# Merge ratings with products
product_ratings = pd.merge(ratings, products, on='product_id', how='inner')


In [5]:
# Merge orders with product_ratings
order_details = pd.merge(orders, product_ratings, on=['customer_id', 'product_id'], how='inner')

# Convert order_date to datetime
order_details['order_date'] = pd.to_datetime(order_details['order_date'])


In [6]:
# Filter orders for the last month
last_month_orders = order_details[order_details['order_date'].dt.month != (pd.Timestamp.now() - pd.DateOffset(months=1)).month]

# Calculate revenue for each product
last_month_orders['revenue'] = last_month_orders['price'] * last_month_orders['quantity']

# Aggregate revenue and units sold for each product
product_performance = last_month_orders.groupby(['product_id', 'product_name']).agg(
    revenue=('revenue', 'sum'),
    units_sold=('quantity', 'sum')
).reset_index()

# Find top performing products in terms of revenue
top_products_revenue = product_performance.sort_values(by='revenue', ascending=False).head(10)

# Find top performing products in terms of units sold
top_products_units_sold = product_performance.sort_values(by='units_sold', ascending=False).head(10)

# Identify top clients for the last month
top_clients = last_month_orders.groupby(['customer_id']).agg(
    total_spent=('revenue', 'sum')
).reset_index().sort_values(by='total_spent', ascending=False).head(10)


In [7]:
# Display results

top_products_revenue_f = pd.DataFrame(top_products_revenue)
print("Top performing products in terms of revenue:") 
top_products_revenue_f.reset_index(drop=True, inplace=True)
top_products_revenue_f


Top performing products in terms of revenue:


,product_id,product_name,revenue,units_sold
0,6,Brimnes Bed Storage,8503,11
1,27,Ivar Cabinet,8244,12
2,44,Koppang Dresser,7650,10
3,12,Raskog Trolley,7640,10
4,31,Nockeby Sofa,7083,9
5,46,Valje Wall Cabinet,6690,10
6,45,Hektar Work Lamp,6456,8
7,40,Lommarp Bookcase,6020,7
8,3,Klippan Loveseat,5751,9
9,5,Hemnes Daybed,5070,6


In [8]:
print("\nTop clients for the last month:")

top_clients_f = pd.DataFrame(top_clients)
top_clients_f = top_clients_f.reset_index(drop=True)
top_clients_f


Top clients for the last month:


,customer_id,total_spent
0,47,6918
1,39,6112
2,8,5536
3,54,5209
4,84,5127
5,7,4921
6,86,4630
7,83,4408
8,38,4388
9,69,4278


In [9]:
print("\nTop performing products in terms of units sold:")
top_products_units_sold_f = pd.DataFrame(top_products_units_sold)
top_products_units_sold_f.reset_index(drop=True, inplace=True)
top_products_units_sold_f



Top performing products in terms of units sold:


,product_id,product_name,revenue,units_sold
0,30,Strandmon Wing Chair,1148,14
1,27,Ivar Cabinet,8244,12
2,6,Brimnes Bed Storage,8503,11
3,44,Koppang Dresser,7650,10
4,46,Valje Wall Cabinet,6690,10
5,12,Raskog Trolley,7640,10
6,31,Nockeby Sofa,7083,9
7,3,Klippan Loveseat,5751,9
8,22,Nordli Chest Drawers,4488,8
9,14,Ingolf Bar Stool,4872,8


### RFM ANALYSIS

In [10]:
# Calculate revenue for each order
order_details['revenue'] = order_details['price'] * order_details['quantity']
# RFM Analysis
# Recency: Calculate the number of days since the last purchase for each customer
recency = order_details.groupby('customer_id')['order_date'].max().reset_index()
# lets check the recency buy days purchsed it from current today date e.g 25 days ago
recency['recency'] = (pd.Timestamp.now() - recency['order_date']).dt.days


In [11]:
# Frequency: Count the number of orders for each customer
frequency = order_details.groupby('customer_id')['order_id'].count().reset_index()
frequency = frequency.rename(columns={'order_id': 'frequency'})

In [12]:
# Monetary: Calculate the total amount spent by each customer
monetary = order_details.groupby('customer_id')['revenue'].sum().reset_index()
monetary = monetary.rename(columns={'revenue': 'monetary'})

In [14]:


# Merge recency, frequency, and monetary dataframes
rfm = pd.merge(recency, frequency, on='customer_id', how='outer')
rfm = pd.merge(rfm, monetary, on='customer_id', how='outer')

# Fill NaN values with 0
rfm = rfm.fillna(0)

# RFM Scores
# Quantile-based scoring
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])
quantiles = quantiles.to_dict()

# Recency score
def recency_score(x):
    if x <= quantiles['recency'][0.25]:
        return 4
    elif x <= quantiles['recency'][0.50]:
        return 3
    elif x <= quantiles['recency'][0.75]:
        return 2
    else:
        return 1

rfm['recency_score'] = rfm['recency'].apply(recency_score)

# Frequency score
def frequency_score(x):
    if x <= quantiles['frequency'][0.25]:
        return 1
    elif x <= quantiles['frequency'][0.50]:
        return 2
    elif x <= quantiles['frequency'][0.75]:
        return 3
    else:
        return 4

rfm['frequency_score'] = rfm['frequency'].apply(frequency_score)

# Monetary score
def monetary_score(x):
    if x <= quantiles['monetary'][0.25]:
        return 1
    elif x <= quantiles['monetary'][0.50]:
        return 2
    elif x <= quantiles['monetary'][0.75]:
        return 3
    else:
        return 4

rfm['monetary_score'] = rfm['monetary'].apply(monetary_score)

# Calculate RFM score
rfm['rfm_score'] = rfm['recency_score'] * 100 + rfm['frequency_score'] * 10 + rfm['monetary_score']

# RFM Segments
# Define RFM segments
def segment(x):
    if x >= 311 and x <= 444:
        return 'Best Customers'
    elif x >= 211 and x < 311:
        return 'Loyal Customers'
    elif x >= 111 and x < 211:
        return 'Potential Loyalists'
    elif x >= 411 and x <= 444:
        return 'Can\'t Lose Them'
    elif x >= 311 and x < 411:
        return 'Promising'
    elif x >= 211 and x < 311:
        return 'Needs Attention'
    elif x >= 111 and x < 211:
        return 'At Risk'
    else:
        return 'Lost'

rfm['segment'] = rfm['rfm_score'].apply(segment)

# Display RFM segments
print("RFM Segments:")
print(rfm.head())
print("\n")

# Analysis and Recommendations
# Analyze each segment and provide recommendations
segment_analysis = rfm.groupby('segment').agg(
    {'recency': 'mean',
     'frequency': 'mean',
     'monetary': 'mean',
     'customer_id': 'count'}
).reset_index()

segment_analysis = segment_analysis.rename(columns={
    'customer_id': 'count',
    'recency': 'average_recency',
    'frequency': 'average_frequency',
    'monetary': 'average_monetary'
})

print("Analysis and Recommendations for each RFM segment:")
segment_analysis


RFM Segments:
   customer_id                 order_date  recency  frequency  monetary  \
0            1 2023-09-10 08:39:23.971834      247          1       717   
1            3 2023-08-16 08:39:23.971834      272          2      1907   
2            4 2023-09-10 08:39:23.971834      247          5      3353   
3            7 2023-08-22 08:39:23.971834      266          3      4921   
4            8 2023-08-27 08:39:23.971834      261          3      5536   

   recency_score  frequency_score  monetary_score  rfm_score          segment  
0              4                1               1        411   Best Customers  
1              2                3               2        232  Loyal Customers  
2              4                4               4        444   Best Customers  
3              2                4               4        244  Loyal Customers  
4              2                4               4        244  Loyal Customers  


Analysis and Recommendations for each RFM segment:


C:\Users\CHRISCODED\AppData\Local\Temp\ipykernel_13784\1183843709.py:10: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])


,segment,average_recency,average_frequency,average_monetary,count
0,Best Customers,249.300000,2.033333,3061.433333,30
1,Loyal Customers,265.692308,1.461538,2324.846154,13
2,Potential Loyalists,286.866667,1.066667,1124.066667,15


## Get top rating

In [16]:
# # Merge ratings with products
# product_ratings = pd.merge(ratings, products, on='product_id', how='inner')
#Merge with product table to get product name
product_ratings = product_ratings.merge(products, on ='product_id')

# Top reviewed items by customers
product_review = product_ratings.groupby(['product_id']).agg(
    review_count=('rating', 'count'),
    average_rating=('rating', 'mean')
).reset_index()

# # Top reviewed items by customers
# product_review = product_ratings.groupby('product_id').agg({'rating':['count', 'mean']}).reset_index()

#get count of rating for a particular product

# product_review = product_ratings.groupby(['product_id', 'product_name'])['rating'].mean().reset_index()

product_review.head()



,product_id,review_count,average_rating
0,1,5,2.800000
1,2,5,3.000000
2,3,6,2.833333
3,4,10,3.900000
4,5,10,2.500000


In [17]:
#flatten the column header for easier reference
# product_review.columns = ['product_id', 'num_reviews', 'avg_rating']

top_reviewed_items = product_review.sort_values(by='average_rating', ascending=False).head(10)

print("Top reviewed items by customers:")
top_reviewed_pd = pd.DataFrame(top_reviewed_items)
top_reviewed_pd


Top reviewed items by customers:


,product_id,review_count,average_rating
24,25,2,4.500000
19,20,6,4.166667
3,4,10,3.900000
43,44,7,3.857143
21,22,5,3.800000
13,14,10,3.800000
36,37,9,3.777778
46,47,4,3.750000
42,43,3,3.666667
49,50,9,3.666667


In [18]:

# Average number of reviews per customer
average_reviews_per_customer = ratings.groupby('customer_id').size().mean()

print("Average number of reviews per customer:", average_reviews_per_customer)

Average number of reviews per customer: 4.040404040404041


In [19]:
# Average rating for each product
average_rating_per_product = product_ratings.groupby(['product_id', 'product_name'])['rating'].mean().reset_index()


average_rating_per_product = pd.DataFrame(average_rating_per_product)
average_rating_per_product



,product_id,product_name,rating
0,1,Ektorp Sofa,2.800000
1,2,Poäng Armchair,3.000000
2,3,Klippan Loveseat,2.833333
3,4,Malm Bed Frame,3.900000
4,5,Hemnes Daybed,2.500000
5,6,Brimnes Bed Storage,3.636364
6,7,Lack Side Table,3.000000
7,8,Melltorp Dining Table,2.750000
8,9,Micke Desk,2.818182
9,10,Kallax Shelving Unit,3.166667


In [44]:
customers2 = pd.read_csv('customers.csv')
products2 = pd.read_csv('products.csv')
ratings2 = pd.read_csv('ratings.csv')
orders2 = pd.read_csv('orders.csv')
# order_details
# Merge orders with products to calculate revenue for each order
orders2 = pd.merge(orders2, products2, on='product_id', how='inner')
orders2['revenue'] = orders2['price'] * orders2['quantity']
orders2

,order_id,customer_id,product_id,order_date,quantity,product_name,price,category,revenue
0,1,66,38,2023-08-09 08:39:23.971834,4,Bekant Conference Table,441,Tables & Desks,1764
1,76,78,38,2023-09-09 08:39:23.971834,4,Bekant Conference Table,441,Tables & Desks,1764
2,102,37,38,2023-07-27 08:39:23.971834,3,Bekant Conference Table,441,Tables & Desks,1323
3,115,47,38,2023-08-17 08:39:23.971834,3,Bekant Conference Table,441,Tables & Desks,1323
4,130,96,38,2023-08-31 08:39:23.971834,1,Bekant Conference Table,441,Tables & Desks,441
...,...,...,...,...,...,...,...,...,...
995,871,79,40,2023-08-31 08:39:23.971834,2,Lommarp Bookcase,860,Storage Solutions,1720
996,915,26,40,2023-08-13 08:39:23.971834,1,Lommarp Bookcase,860,Storage Solutions,860
997,927,27,40,2023-08-22 08:39:23.971834,3,Lommarp Bookcase,860,Storage Solutions,2580
998,942,39,40,2023-08-05 08:39:23.971834,2,Lommarp Bookcase,860,Storage Solutions,1720


In [52]:
# ccc = ratings2.groupby('customer_id').size().reset_index(name='review_count')
# ccc

In [61]:


# Calculate total amount spent and last purchase date for each customer
customer_orders = orders2.groupby('customer_id').agg(
    total_spent=('revenue', 'sum'),
    last_purchase_date=('order_date', 'max')
).reset_index()

# Calculate the number of reviews left by each customer
customer_reviews = ratings2.groupby('customer_id').size().reset_index(name='review_count')

# Merge the new data with the customers table
customers = pd.merge(customers, customer_orders, on='customer_id', how='left')
customers = pd.merge(customers, customer_reviews, on='customer_id', how='left')

# Fill NaN values with 0 for 'review_count' and 'total_spent', and the minimum possible date for 'last_purchase_date'
customers['total_spent'] = customers['total_spent'].fillna(0)
customers['review_count'] = customers['review_count'].fillna(0)
customers['last_purchase_date'] = customers['last_purchase_date'].fillna(pd.Timestamp.min)

# Display the updated customers table
print("Updated customers table:")
customers = pd.DataFrame(customers)
customers

Updated customers table:


,customer_id,name,total_spent_x,last_purchase_date_x,review_count_x,total_spent_y,last_purchase_date_y,review_count_y,total_spent_x,last_purchase_date_x,...,review_count_y,total_spent_x,last_purchase_date_x,review_count_x,total_spent_y,last_purchase_date_y,review_count_y,total_spent,last_purchase_date,review_count
0,1,Customer_1,27081,2023-09-17 08:39:23.971834,4.0,27081,2023-09-17 08:39:23.971834,4.0,27081,2023-09-17 08:39:23.971834,...,4.0,27081,2023-09-17 08:39:23.971834,4.0,27081,2023-09-17 08:39:23.971834,4.0,27081,2023-09-17 08:39:23.971834,4.0
1,2,Customer_2,9347,2023-09-15 08:39:23.971834,2.0,9347,2023-09-15 08:39:23.971834,2.0,9347,2023-09-15 08:39:23.971834,...,2.0,9347,2023-09-15 08:39:23.971834,2.0,9347,2023-09-15 08:39:23.971834,2.0,9347,2023-09-15 08:39:23.971834,2.0
2,3,Customer_3,18077,2023-09-13 08:39:23.971834,4.0,18077,2023-09-13 08:39:23.971834,4.0,18077,2023-09-13 08:39:23.971834,...,4.0,18077,2023-09-13 08:39:23.971834,4.0,18077,2023-09-13 08:39:23.971834,4.0,18077,2023-09-13 08:39:23.971834,4.0
3,4,Customer_4,25717,2023-09-14 08:39:23.971834,7.0,25717,2023-09-14 08:39:23.971834,7.0,25717,2023-09-14 08:39:23.971834,...,7.0,25717,2023-09-14 08:39:23.971834,7.0,25717,2023-09-14 08:39:23.971834,7.0,25717,2023-09-14 08:39:23.971834,7.0
4,5,Customer_5,14913,2023-09-09 08:39:23.971834,2.0,14913,2023-09-09 08:39:23.971834,2.0,14913,2023-09-09 08:39:23.971834,...,2.0,14913,2023-09-09 08:39:23.971834,2.0,14913,2023-09-09 08:39:23.971834,2.0,14913,2023-09-09 08:39:23.971834,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Customer_96,12264,2023-09-15 08:39:23.971834,4.0,12264,2023-09-15 08:39:23.971834,4.0,12264,2023-09-15 08:39:23.971834,...,4.0,12264,2023-09-15 08:39:23.971834,4.0,12264,2023-09-15 08:39:23.971834,4.0,12264,2023-09-15 08:39:23.971834,4.0
96,97,Customer_97,6088,2023-09-09 08:39:23.971834,2.0,6088,2023-09-09 08:39:23.971834,2.0,6088,2023-09-09 08:39:23.971834,...,2.0,6088,2023-09-09 08:39:23.971834,2.0,6088,2023-09-09 08:39:23.971834,2.0,6088,2023-09-09 08:39:23.971834,2.0
97,98,Customer_98,8915,2023-09-11 08:39:23.971834,5.0,8915,2023-09-11 08:39:23.971834,5.0,8915,2023-09-11 08:39:23.971834,...,5.0,8915,2023-09-11 08:39:23.971834,5.0,8915,2023-09-11 08:39:23.971834,5.0,8915,2023-09-11 08:39:23.971834,5.0
98,99,Customer_99,13181,2023-09-17 08:39:23.971834,4.0,13181,2023-09-17 08:39:23.971834,4.0,13181,2023-09-17 08:39:23.971834,...,4.0,13181,2023-09-17 08:39:23.971834,4.0,13181,2023-09-17 08:39:23.971834,4.0,13181,2023-09-17 08:39:23.971834,4.0


#